## CPSC 340 Lecture 21: kernel methods

This notebook is for the in-class activities. It assumes you have already watched the [associated video](https://www.youtube.com/watch?v=mba4xweShwI&list=PLWmXHcz_53Q02ZLeAxigki1JZFfCO6M-b&index=22).

<font color='red'>**REMINDER TO START RECORDING**</font>

Also, reminder to enable screen sharing for Participants.

## Pre-class music



1. Sweet Baby! by Mere Notilde
2. THE BADDEST by K/DA

## Admin

- a4 due Wed

## Video chapters

- linear SVM minutiae
- n-by-n normal equations
- feature transformation demo
- multi-dimensional polynomial basis
- kernel trick
- linear regression vs. kernel regression
- Gaussian RBF kernel
- kernel trick applications
- RBF SVM hyperparameters demo
- RBF SVM minutiae
- summary

## Old exam questions

#### 2016W2 final

Name an advantage and a disadvantage of using an SVM with the RBF kernel instead of a linear kernel.

#### 2016W2 final

![](img/L21/matching_top.png)

![](img/L21/matching_bottom.png)

#### 2018W1 final

![](img/L21/matching2_top.png)

![](img/L21/matching2_bottom.png)